<br><br>

## **Import necessary Python libraries and modules**

In [1]:

# For data manipulation and analysis
import pandas as pd
import numpy as np

# For machine learning tools and evaluation
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report


# For deep learning
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
import torch



To use the HuggingFace [`transformers` Python library](https://huggingface.co/transformers/installation.html), we will install it with `pip`.

In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 12.8 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 6.5 MB 41.5 MB/s 
     |████████████████████████████████| 895 kB 42.7 MB/s 
     |████████████████████████████████| 596 kB 48.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Once `transformers` is installed, we will import modules for `DistilBert`, a *distilled* or smaller version of a BERT model that runs more quickly and uses less computing power. This makes it ideal for those just getting started with BERT.

In [3]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

<br><br>

## **Set parameters and file paths**

In [5]:
# This is the name of the BERT model that we want to use. 
# We're using DistilBERT to save space (it's a distilled version of the full BERT model), 
# and we're going to use the cased (vs uncased) version.
model_name = 'distilbert-base-multilingual-cased'  

# This is the name of the program management system for NVIDIA GPUs. We're going to send our code here.
device_name = 'cuda'       

# This is the maximum number of tokens in any document sent to BERT.
max_length = 128                                                       

# This is the name of the directory where we'll save our model. You can name it whatever you want.
#cached_model_directory_name = 'ABSA_FineTuning_BERT'  
cached_model_directory_name = 'Pol_dev2'  

In [6]:
#stiamo utlizzando la GPU?
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [7]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 6.5 MB/s 
     |████████████████████████████████| 448 kB 43.5 MB/s 


In [8]:
#pretokenizzazionen in morfemi delle koreane
from konlpy.tag import Okt
def pretok_ko(subset, indice):
  new_ko=[] 
  #count=0
  for index,row in subset.iterrows():
    if index < indice:
      new_ko.append(row['Sentence'])
    else: #quando arrivi all'indice di ko

      okt = Okt()
      li=okt.morphs(str(row['Sentence']))
      stringa=''
      for el in li:
        stringa+=str(el)+' '
      new_ko.append(stringa[:-1])
  subset['Sentence']=new_ko
  return subset
    


In [9]:
dev=pd.read_csv("dev_def_pol.csv")
dev=pretok_ko(dev,352)
dev

,Unnamed: 0,Sentence,Polarity_Classification
0,0,The solution given here is fantastic and a bit...,Positiva
1,1,It is well worth your time to read this one,Positiva
2,2,This book is jam-packed with many fantastical ...,Positiva
3,3,The European locations never quite come to lif...,Negativa
4,4,But the biggest problem with The Last Temptati...,Negativa
...,...,...,...
694,694,나 는 왜 이 영화 가 소통 부재 에 대한 썰로 이리 유명한지 모르겠다 . 막상 영...,Negativa
695,695,강풀 원작 영화 이웃사람 을 보고나니 문득 예전 에 봤던 이 영화 가 떠올랐다 . ...,Negativa
696,696,누를 땐 니 맘 일지 몰라도 결과 는 모른다하 경준 씨,Negativa
697,697,노잼 노잼 인터스텔라 보다 별로 라는 앤 뭐 지 ? 그냥 놀래 키 는 거 랑 잔인하...,Negativa


In [10]:
test=pd.read_csv("test_def_pol.csv")
test=pretok_ko(test,699)
test

,Unnamed: 0,Sentence,Polarity_Classification
0,0,This is one of the books that my dad read out ...,Positiva
1,1,Garth Nix's 'Mister Monday' was a highly cleve...,Positiva
2,2,"Includes affirmations, visualizations, and pra...",Positiva
3,3,Monday was fantasy,Neutra
4,4,Weird and dangerous creatures are hunting him ...,Neutra
...,...,...,...
1387,1387,"달리는 동안 그 는 이 것 을 작은 열쇠 라고 하는 것 을 갖게 되었고 , 그것 은...",Neutra
1388,1388,아벨라 부터 들어간 기아 순정 이긴 한데 프라이드 는 잘 모르 겠 네요 ㅋㅋ 끝물 ...,Neutra
1389,1389,이전 이나 변함 없다 .,Negativa
1390,1390,[ 3 점 ] 뭐 하자는거지 ?,Negativa


In [11]:
#creiamo le liste di train e test. 1.8k train 250 test  
train_texts=dev['Sentence']
train_labels=dev['Polarity_Classification']
test_texts=test['Sentence']
test_labels=test['Polarity_Classification']


<br><br>

## **Split the data into training and test sets**

In [12]:
len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(699, 699, 1392, 1392)

Here's an example of a training label and review:

In [13]:
train_labels[0], train_texts[0]

('Positiva',
 'The solution given here is fantastic and a bit bizarre and there lies the appeal of this book')

<br><br>

## **Encode data for BERT**

We're going to transform our texts and labels into a format that BERT (via Huggingface and PyTorch) will understand. This is called *encoding* the data.

Here are the steps we need to follow:

1. The labels&mdash;in this case, Goodreads genres&mdash;need to be turned into integers rather than strings.

2. The texts&mdash;in this case, Goodreads reviews&mdash;need to be truncated if they're more than 512 tokens or padded if they're fewer than 512 tokens. The tokens, or words in the texts, also need to be separated into "word pieces" and matched to their embedding vectors.

3. We need to add special tokens to help BERT:

| BERT special token | Explanation |
| --------------| ---------|
| [CLS] | Start token of every document. |
| [SEP] | Separator between each sentence |
| [PAD] | Padding at the end of the document as many times as necessary, up to 512 tokens |
|  &#35;&#35; | Start of a "word piece" |




Here we will load `DistilBertTokenizerFast` from the HuggingFace library, which will do all the work of encoding the texts for us. The `tokenizer()` will break word tokens into word pieces, truncate to 512 tokens, and add padding and special BERT tokens.

In [14]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Here we will create a map of our labels, or Goodreads genres, to integer keys. We take the unique labels, and then we make a dictionary that associates each label/tag with an integer.

**Note:** HuggingFace documentation sometimes refers to "labels" as "tags" but these are the same thing. We use "labels" throughout this notebook for clarity.

In [15]:
#unique_labels = set(label for label in train_labels)
#label2id = {label: id for id, label in enumerate(unique_labels)}
#id2label = {id: label for label, id in label2id.items()}

In [16]:
unique_labels={'Neutra', 'Positiva', 'Negativa'}
label2id={'Negativa': 2, 'Neutra': 0, 'Positiva': 1}
id2label={2:'Negativa', 0:'Neutra', 1:'Positiva'}

In [17]:
unique_labels

{'Negativa', 'Neutra', 'Positiva'}

In [18]:
label2id.keys()

dict_keys(['Negativa', 'Neutra', 'Positiva'])

In [19]:
id2label.keys()

dict_keys([2, 0, 1])

Now let's encode our texts and labels!

In [20]:
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
test_encodings  = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)

train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]

In [21]:
type(test_labels_encoded[100])

int

In [22]:
' '.join(train_encodings[29].tokens[0:100])

'[CLS] Arthur Pen ##hali ##gon and Su ##zy Blue are two ama ##zing characters , Arthur : a normal boy , as ##th ##mati ##c , not too con ##fi ##dent about himself , intelligent and thought ##ful [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [23]:
' '.join(test_encodings[0].tokens[0:100])

'[CLS] This is one of the books that my dad read out lo ##ud to me at bed ##time , but when ##ever he had to put it down , I begge ##d him to read more [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [24]:
set(train_labels_encoded)

{0, 1, 2}

In [25]:
set(test_labels_encoded)

{0, 1, 2}

<br><br>

## **Make a custom Torch dataset**

Here we combine the encoded labels and texts into dataset objects. We use the custom Torch `MyDataSet` class to make a `train_dataset` object from  the `train_encodings` and `train_labels_encoded`. We also make a `test_dataset` object from `test_encodings`, and `test_labels_encoded`.

In [26]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [27]:
train_dataset = MyDataset(train_encodings, train_labels_encoded)
test_dataset = MyDataset(test_encodings, test_labels_encoded)

In [28]:
' '.join(train_dataset.encodings[0].tokens[0:100])

'[CLS] The solution given here is fant ##astic and a bit biz ##arre and there lies the appeal of this book [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [29]:
' '.join(test_dataset.encodings[1].tokens[0:100])

"[CLS] Ga ##rth Ni ##x ' s ' Mister Monday ' was a highly c ##lever , creative , and enter ##taining read that had me up into the we ##e hours of the morning following Arthur ' s adventure ##s in the House [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

<br><br>

## **Load pre-trained BERT model**

Here we load a pre-trained DistilBERT model and send it to CUDA.

**Note:** If you decide to repeat fine-tuning after already running the following cells, make sure that you re-run this cell to re-load the original pre-trained model before fine-tuning again.

In [30]:
# The model_name needs to match the name used for the tokenizer above.
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label)).to(device_name)

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.bias', 'classifie

<br><br>

## **Set the BERT fine-tuning parameters**

These are the arguments we'll set in the HuggingFace TrainingArguments objects, which we'll then pass to the HuggingFace Trainer object. There are many more possible arguments, but here we highlight the basics and some common gotchas.

When training your own model, you should search over these parameters to find the best settings for your particular dataset. You should use a held-out set of validation data for this step.

| Parameter | Explanation |
|-----------| ------------|
| num_train_epochs | total number of training epochs (how many times to pass through the entire dataset; too much can cause overfitting) |
| per_device_train_batch_size | batch size per device during training |
| per_device_eval_batch_size |  batch size for evaluation |
|  warmup_steps |  number of warmup steps for learning rate scheduler (set lower because of small dataset size) |
| weight_decay | strength of weight decay (reduces size of weights, like regularization) |
| output_dir | output directory for the fine-tuned model and configuration files |
| logging_dir | directory for storing logs |
| logging_steps | how often to print logging output (so that we can stop training early if the loss isn't going down) |
| evaluation_strategy | evaluate while training so that we can see the accuracy going up |

In [31]:
training_args = [TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    learning_rate=5e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'
    # evaluate during fine-tuning so that we can see progress
), TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'# evaluate during fine-tuning so that we can see progress
), 
TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    learning_rate=3e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'# evaluate during fine-tuning so that we can see progress
),
TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=5e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'# evaluate during fine-tuning so that we can see progress
), 
TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'# evaluate during fine-tuning so that we can see progress
),
TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=3e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'# evaluate during fine-tuning so that we can see progress
)]

<br><br>

## **Fine-tune the BERT model**

First, we define a custom evaluation function that returns the accuracy. You could modify this function to return precision, recall, F1, and/or other metrics.

In [32]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

Then we create a HuggingFace `Trainer` object using the `TrainingArguments` object that we created above. We also send our `compute_metrics` function to the `Trainer` object, along with our test and train datasets.

**Note:** This is what we've been aiming for this whole time! All the work of tokenizing, creating datasets, and setting the training arguments was for this cell.

In [33]:
for argomenti_train in training_args:
  trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=argomenti_train,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset (usually a validation set; here we just send our test set)
    compute_metrics=compute_metrics      # our custom evaluation function 
    )
  
  trainer.train()

  trainer.evaluate()   

  predicted_results = trainer.predict(test_dataset)

  predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
  predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
  predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

  print(classification_report(test_labels, 
                            predicted_labels))
  print('\n')


#meglio prediligere learn rate bassi ad ogni modo
#https://arxiv.org/pdf/1905.05583.pdf

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 699
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 88


Step,Training Loss,Validation Loss


Saving model checkpoint to ./results/checkpoint-22
Configuration saved in ./results/checkpoint-22/config.json
Model weights saved in ./results/checkpoint-22/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-44
Configuration saved in ./results/checkpoint-44/config.json
Model weights saved in ./results/checkpoint-44/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-66
Configuration saved in ./results/checkpoint-66/config.json
Model weights saved in ./results/checkpoint-66/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-88
Configuration saved in ./results/checkpoint-88/config.json
Model weights saved in ./results/checkpoint-88/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 1392
  Batch size = 32


***** Running Prediction *****
  Num examples = 1392
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 699
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 88


              precision    recall  f1-score   support

    Negativa       0.53      0.58      0.56       460
      Neutra       0.53      0.70      0.61       467
    Positiva       0.41      0.24      0.30       465

    accuracy                           0.51      1392
   macro avg       0.49      0.51      0.49      1392
weighted avg       0.49      0.51      0.49      1392





...

              precision    recall  f1-score   support

    Negativa       0.57      0.50      0.53       460
      Neutra       0.67      0.74      0.70       467
    Positiva       0.55      0.57      0.56       465

    accuracy                           0.60      1392
   macro avg       0.60      0.60      0.60      1392
weighted avg       0.60      0.60      0.60      1392





Step,Training Loss,Validation Loss


Saving model checkpoint to ./results/checkpoint-44
Configuration saved in ./results/checkpoint-44/config.json
Model weights saved in ./results/checkpoint-44/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-88
Configuration saved in ./results/checkpoint-88/config.json
Model weights saved in ./results/checkpoint-88/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-132
Configuration saved in ./results/checkpoint-132/config.json
Model weights saved in ./results/checkpoint-132/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-176
Configuration saved in ./results/checkpoint-176/config.json
Model weights saved in ./results/checkpoint-176/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 1392
  Batch size = 16


***** Running Prediction *****
  Num examples = 1392
  Batch size = 16


              precision    recall  f1-score   support

    Negativa       0.53      0.45      0.49       460
      Neutra       0.71      0.64      0.67       467
    Positiva       0.53      0.65      0.58       465

    accuracy                           0.58      1392
   macro avg       0.59      0.58      0.58      1392
weighted avg       0.59      0.58      0.58      1392





In [34]:
#anche qui il compound 5 è top